In [2]:
import os
import sqlalchemy
import numpy as np
import pandas as pd

In [3]:
THIS_DIR  = os.getcwd()
WORK_DIR  = os.path.dirname(os.path.dirname(THIS_DIR))
DATA_DIR  = os.path.join(WORK_DIR, 'data')

In [4]:
string_connection = 'sqlite:///{path}'.format(path=os.path.join(DATA_DIR, 'churn.sqlite'))
connection = sqlalchemy.create_engine(string_connection)

In [5]:
df = pd.read_sql('SELECT * FROM abt', con=connection)

In [6]:
df.head()

,user_id,nb_products_seen,nb_distinct_product,nb_distinct_cat_0,nb_distinct_cat_1,nb_distinct_cat_2,amount_bought,nb_product_bought,active_time,target
0,1,16,16,7,11,11,158.26,3,123,1
1,143,8,8,4,6,6,43.11,1,105,0
2,164,234,216,10,58,58,4154.21,97,200,0
3,198,6,6,3,6,6,76.95,3,38,0
4,242,23,20,7,14,14,58.08,4,201,0


In [7]:
df.shape

(4602, 10)

In [8]:
df['target'].value_counts(normalize=True)

1    0.762929
0    0.237071
Name: target, dtype: float64

In [14]:
df.dtypes

user_id                  int64
nb_products_seen         int64
nb_distinct_product      int64
nb_distinct_cat_0        int64
nb_distinct_cat_1        int64
nb_distinct_cat_2        int64
amount_bought          float64
nb_product_bought        int64
active_time              int64
target                   int64
dtype: object

### Modeling with Pycaret

In [9]:
from pycaret.classification import *

In [10]:
churn_setup = setup(data=df, target='target', ignore_features=['user_id'], train_size=0.8, session_id=42)


Setup Succesfully Completed!


,Description,Value
0,session_id,42
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(4602, 10)"
4,Missing Values,False
5,Numeric Features,8
6,Categorical Features,1
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [13]:
compare_models(fold=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Gradient Boosting Classifier,0.816900,0.820500,0.937000,0.841100,0.886400,0.420900
1,Extreme Gradient Boosting,0.816600,0.821500,0.936600,0.841100,0.886300,0.420300
2,CatBoost Classifier,0.813100,0.813300,0.928800,0.842400,0.883400,0.417400
3,Ridge Classifier,0.811700,0.000000,0.938800,0.835000,0.883800,0.396600
4,Ada Boost Classifier,0.808200,0.811100,0.928800,0.837500,0.880700,0.397600
5,Linear Discriminant Analysis,0.807700,0.818200,0.920600,0.842000,0.879500,0.407200
6,Light Gradient Boosting Machine,0.805500,0.798000,0.917000,0.842100,0.877900,0.403600
7,K Neighbors Classifier,0.797100,0.752000,0.913800,0.835600,0.872900,0.374100
8,Extra Trees Classifier,0.794400,0.787100,0.903100,0.839500,0.870100,0.379700
9,Random Forest Classifier,0.786500,0.777500,0.873900,0.850400,0.861800,0.391400


In [16]:
xgb_model = create_model('xgboost', fold=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.8046,0.7956,0.9199,0.8393,0.8778,0.3963
1,0.8329,0.8517,0.9483,0.8498,0.8964,0.4714
2,0.8207,0.8337,0.9447,0.8399,0.8893,0.4263
3,0.8179,0.8292,0.9377,0.8419,0.8872,0.4224
4,0.8071,0.7972,0.9324,0.8344,0.8807,0.3851
Mean,0.8166,0.8215,0.9366,0.8411,0.8863,0.4203
SD,0.0102,0.0218,0.0100,0.0050,0.0066,0.0299


In [17]:
tuned_xgb_model = tune_model('xgboost', fold=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.7910,0.7680,0.9075,0.8333,0.8688,0.3600
1,0.8139,0.8253,0.9002,0.8618,0.8806,0.4599
2,0.8179,0.8260,0.9109,0.8588,0.8841,0.4617
3,0.7976,0.7845,0.9128,0.8369,0.8732,0.3762
4,0.7677,0.7544,0.8861,0.8231,0.8535,0.2965
Mean,0.7976,0.7916,0.9035,0.8428,0.8720,0.3909
SD,0.0180,0.0294,0.0097,0.0150,0.0107,0.0630
